# Bien penser à changer le CSV et le nom des DF !!

In [1]:
from selenium import webdriver
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import pandas as pd
import re, time, random

In [2]:
df_data_villes_clean = pd.read_csv("pays_de_la_loire_V2.csv", 
                                    delimiter=",",
                                    dtype=str,
                                    index_col=0)


In [3]:
df_data_villes_clean.head()


,#Code_commune_INSEE,Lib_MA,departement,Code_postal,Nom_de_la_commune_url,url
17918,44001,ABBARETZ,44,44170,abbaretz,https://www.meilleursagents.com/prix-immobilie...
17919,44002,AIGREFEUILLE SUR MAINE,44,44140,aigrefeuille-sur-maine,https://www.meilleursagents.com/prix-immobilie...
17920,44003,ANCENIS,44,44150,ancenis,https://www.meilleursagents.com/prix-immobilie...
17921,44003,ST GEREON,44,44150,saint-gereon,https://www.meilleursagents.com/prix-immobilie...
17922,44005,ARTHON EN RETZ,44,44320,arthon-en-retz,https://www.meilleursagents.com/prix-immobilie...


In [4]:
def extract_number(text):
    """Nettoyer le texte et transformer en float"""
    clean_text = text.replace("\u202f", "").replace("\xa0", "").replace("€", "").strip()
    clean_text = clean_text.replace(",", ".")
    return float(clean_text)

# Selenium setup (Chrome auto-managed by Selenium Manager; Chrome must be installed)
options = webdriver.ChromeOptions()
options.add_argument("--lang=fr-FR")
options.add_argument("--disable-blink-features=AutomationControlled")
# Uncomment to persist cookies/session between runs:
# options.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Local\Google\Chrome\User Data\ScrapeProfile")

driver = webdriver.Chrome(options=options)
stealth(driver,
    languages=["fr-FR", "fr"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)
driver.set_window_size(1366, 768)

# Lancer le navigateur
driver.get("https://www.meilleursagents.com/prix-immobilier/")

In [5]:
# Liste pour stocker toutes les données
all_data = []
error_data = []

for index, row in df_data_villes_clean.iterrows():
    url = row['url']
    code_insee = row['#Code_commune_INSEE']
    try:
        # Human-like pacing
        time.sleep(random.uniform(1.8, 3.6))

        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        
        data_dict = {}
        # 1-Ajouter la ville
        title_text = soup.find("title").get_text().strip()
        ville = title_text.split("Prix immobilier")[1].split("(")[0].strip()
        data_dict["ville"] = ville
        
        #2-ajouter prix achat+loyer au m2 appt et maison
        raw_text3 = soup.find_all("ul", class_="prices-summary__price-range")
        for ul in raw_text3:
            li_items = ul.find_all("li")
            label = li_items[0].get_text().strip().lower()
            main_value = extract_number(li_items[1].get_text())
            # Nettoyer le texte pour récupérer min et max correctement
            range_text = li_items[2].get_text()
            range_text = range_text.replace("\u202f", "").replace("\xa0", "").replace(",", ".")
            # Extraire tous les nombres sous forme de float
            numbers_in_range = [float(n) for n in re.findall(r"[\d\.]+", range_text)]
            if "prix" in label:
                if "prix_appartement" not in data_dict:
                    data_dict["prix_appartement"] = main_value
                    data_dict["min_appartement"] = numbers_in_range[0]
                    data_dict["max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["prix_maison"] = main_value
                    data_dict["min_maison"] = numbers_in_range[0]
                    data_dict["max_maison"] = numbers_in_range[1]
            elif "loyer" in label:
                if "loyer_appartement" not in data_dict:
                    data_dict["loyer_appartement"] = main_value
                    data_dict["loyer_min_appartement"] = numbers_in_range[0]
                    data_dict["loyer_max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["loyer_maison"] = main_value
                    data_dict["loyer_min_maison"] = numbers_in_range[0]
                    data_dict["loyer_max_maison"] = numbers_in_range[1]
        
        #3-Ajouter l'URL pour référence
        data_dict["url"] = url
        data_dict['Code_insee'] = code_insee
        all_data.append(data_dict)
        print(f"Datas ajoutées pour la ville : {ville}")
        
    except Exception as erreur:
        print(f"Erreur pour l'URL {url}: {erreur}")
        error_data.append({
        "url": url,
        "erreur": str(erreur)})
        
    # Pause pour ne pas surcharger le site
    time.sleep(random.uniform(2.3, 3.6))

driver.quit()



Datas ajoutées pour la ville : Abbaretz
Datas ajoutées pour la ville : Aigrefeuille-sur-Maine
Datas ajoutées pour la ville : Ancenis
Datas ajoutées pour la ville : Saint-Géréon
Datas ajoutées pour la ville : Chaumes-en-Retz
Datas ajoutées pour la ville : La Sicaudais
Datas ajoutées pour la ville : Chaumes-en-Retz
Datas ajoutées pour la ville : Assérac
Datas ajoutées pour la ville : Pont d'Armes
Datas ajoutées pour la ville : Avessac
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/basse-goulaine-44115/: list index out of range
Datas ajoutées pour la ville : Batz-sur-Mer
Datas ajoutées pour la ville : La Bernerie-en-Retz
Datas ajoutées pour la ville : Besné
Datas ajoutées pour la ville : Le Bignon
Datas ajoutées pour la ville : Blain
Datas ajoutées pour la ville : Saint Emilien de Blain
Datas ajoutées pour la ville : Saint Omer de Blain
Datas ajoutées pour la ville : La Boissière-du-Doré
Datas ajoutées pour la ville : Bouaye
Datas ajoutées pour la ville : Bouée
Erreur p

In [7]:
df_error_url = pd.DataFrame(error_data)
df_meilleur_agent_PdlL = pd.DataFrame(all_data)

In [8]:
df_meilleur_agent_PdlL.head()

,ville,prix_appartement,min_appartement,max_appartement,prix_maison,min_maison,max_maison,loyer_appartement,loyer_min_appartement,loyer_max_appartement,loyer_maison,loyer_min_maison,loyer_max_maison,url,Code_insee
0,Abbaretz,1715.0,1006.0,2836.0,1704.0,682.0,2935.0,12.0,8.3,19.7,11.7,7.6,15.0,https://www.meilleursagents.com/prix-immobilie...,44001
1,Aigrefeuille-sur-Maine,2704.0,1673.0,3682.0,2630.0,1052.0,3875.0,13.0,9.6,17.0,12.7,8.8,15.5,https://www.meilleursagents.com/prix-immobilie...,44002
2,Ancenis,2965.0,1629.0,4876.0,2730.0,1230.0,4376.0,11.8,7.9,19.5,12.4,8.0,16.3,https://www.meilleursagents.com/prix-immobilie...,44003
3,Saint-Géréon,2907.0,1649.0,4184.0,2504.0,1183.0,3677.0,12.4,8.2,17.7,12.2,9.4,16.1,https://www.meilleursagents.com/prix-immobilie...,44003
4,Chaumes-en-Retz,2533.0,1474.0,3650.0,2412.0,965.0,3581.0,13.5,9.4,21.2,12.5,8.9,15.6,https://www.meilleursagents.com/prix-immobilie...,44005


In [9]:
print(df_meilleur_agent_PdlL.shape)
print(df_error_url.shape)

(1030, 15)
(572, 2)


In [25]:
df_duplicate = df_meilleur_agent5.drop_duplicates(subset=['ville'])

In [10]:
df_meilleur_agent_PdlL.to_csv('ma_1.csv',index=False)
df_error_url.to_csv('error_ma_1.csv',index=False)

In [47]:
df_meilleur_agent9['ratio_appt'] = round((df_meilleur_agent9['prix_appartement'] / df_meilleur_agent9['loyer_appartement']),1)

df_meilleur_agent9['ratio_maison'] = round((df_meilleur_agent9['prix_maison'] / df_meilleur_agent9['loyer_maison']),1)

df_meilleur_agent9 = df_meilleur_agent9.sort_values('ratio_appt')